In [170]:
import numpy as np
import pandas as pd
import os

def decimal2binary(dec_number):
    input_binary = bin(dec_number).replace("0b", "")
    #TM_binary = str('110') + input_binary + str('110')
    return input_binary
def expansion(n):
	n=decimal2binary(n)
	n=[int(x) for x in (n)]
	n2=[]
	n2.append(0)
	while len(n)!=0:
		if n[0]==0 & (len(n) >= 1):
			n2.append(0)
			del n[0]
		elif (n[0]==1) & (len(n) >= 1):
			n2.append(1)
			n2.append(0)
			del n[0]
		#print(n)
	n2.append(1)
	n2.append(1)
	n2.append(0)
	return n2

In [171]:
a=pd.read_csv("procesado.csv", header=None)
a.columns=(["Si","Si+1","Read","Write","Move"])
a

,Si,Si+1,Read,Write,Move
0,0,0.0,0,0,R
1,0,10000000.0,1,1,L
2,1,1.0,0,0,R
3,1,1001011.0,1,1,R
4,10,10.0,0,0,R
...,...,...,...,...,...
397,11000110,1100111.0,1,1,L
398,11000111,NaN,0,0,R
399,11000111,11001.0,1,1,R
400,11001000,100.0,0,0,R


In [151]:
a.iloc[:]["Move"]=="H" #Verificación de que la MT ha parado

0    False
1     True
Name: Move, dtype: bool

In [152]:
a["Move"].where(a["Move"]=="H").index

RangeIndex(start=0, stop=2, step=1)

In [98]:
#este código que está por aquí 
idx = (a.index[a['Read']==1] & a.index[a["Move"]=="H"])
a.loc[idx]["Move"][1]

C:\Users\emman\AppData\Local\Temp\ipykernel_26372\3331589139.py:2: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  idx = (a.index[a['Read']==1] & a.index[a["Move"]=="H"])


'H'

In [153]:
idx=(a.index[a['Read']==0] & a.index[a["Si"]==0])
idx

C:\Users\emman\AppData\Local\Temp\ipykernel_26372\521132656.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  idx=(a.index[a['Read']==0] & a.index[a["Si"]==0])


Int64Index([0], dtype='int64')

In [154]:
estado=a.loc[idx]["Si+1"].item()
estado

0

In [193]:
def decode(instrucciones, n):
    cinta=expansion(n)
    estado=0
    posicion=0
    grafo={}
    while estado!="H":
        if (cinta[posicion]==0):
            idx=(instrucciones.index[instrucciones['Read']==0] & instrucciones.index[instrucciones["Si"]==estado])
            #print(idx)
            estado=instrucciones.loc[idx]["Si+1"].item()
            #print(estado)
            cinta[posicion]=instrucciones.loc[idx]["Write"].item()
            if instrucciones.loc[idx]["Move"].item()=="H":
                posicion=posicion
                break
            elif instrucciones.loc[idx]["Move"].item()=="R":
                posicion=posicion+1
            elif instrucciones.loc[idx]["Move"].item()=="L":
                posicion=posicion-1  
        elif (cinta[posicion]==1):
            idx=(instrucciones.index[instrucciones['Read']==1] & instrucciones.index[instrucciones["Si"]==estado])
            #print(idx)
            estado=instrucciones.loc[idx]["Si+1"].item()
            #print("muy buenas noches")
            cinta[posicion]=instrucciones.loc[idx]["Write"].item()
            if instrucciones.loc[idx]["Move"].item()=="H":
                posicion=posicion
                #print(cinta)
                break
            elif instrucciones.loc[idx]["Move"].item()=="R":
                posicion=posicion+1
            elif instrucciones.loc[idx]["Move"].item()=="L":
                posicion=posicion-1
                #print(cinta)
        print(posicion)
    return cinta

In [198]:
decode(a, 21312)

C:\Users\emman\AppData\Local\Temp\ipykernel_26372\664024537.py:8: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  idx=(instrucciones.index[instrucciones['Read']==0] & instrucciones.index[instrucciones["Si"]==estado])
C:\Users\emman\AppData\Local\Temp\ipykernel_26372\664024537.py:21: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  idx=(instrucciones.index[instrucciones['Read']==1] & instrucciones.index[instrucciones["Si"]==estado])


1
0
-1
-2
-1
0
1
2
1
0
-1
-2
-3
-4
-5
-6
-7
-8
-9
-10
-11
-12
-13
-14
-15
-16
-17
-18
-19
-20
-21
-22
-23
-24
-25


IndexError: list index out of range

In [175]:
expansion(1)

[0, 1, 0, 1, 1, 0]

In [166]:
a

,Si,Si+1,Read,Write,Move
0,0,0,0,0,R
1,0,1,1,1,H
